[View in Colaboratory](https://colab.research.google.com/github/wangtianrui/Game/blob/master/pinganGame1.ipynb)

In [5]:
from google.colab import files

uploaded = files.upload()

Saving wang_data.csv to wang_data.csv
Saving wang_data_test.csv to wang_data_test.csv


In [6]:
!ls

datalab  wang_data.csv	wang_data_test.csv


In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import os

num_class = 2
batch_size = 100
train_path = r"wang_data.csv"
test_path = r"wang_data_test.csv"
learning_rate = 0.0000001
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
size = 37
log_path = r"./log/"


def readData(path):
    whole_data = pd.read_csv(path, encoding='utf-8')
    whole_data = whole_data.fillna(0)
    whole_list = np.array(whole_data)
    for i in range(len(whole_list)):
        for temp in range(len(whole_list[i])):
            if whole_list[i][temp] == " ":
                whole_list[i][temp] = 0
    # print("转换前：", whole_list)
    whole_list = np.array(whole_list, dtype="float")
    # print(whole_list)
    return whole_list


def makeData(data, flag):
    x = []
    y = []
    ran = []
    # print(len(data))
    # print(len(data[1480]))
    # print(data)
    if flag == "train":
        for i in range(batch_size):
            index = random.randint(0, 6384)
            # print("test data len", len(data))
            row = data[index]
            label = [int(row[37])]
            y.append(label)
            row = np.delete(row, 37)
            x.append(row)
            ran.append(index)
    else:
        for i in range(batch_size):
            index = random.randint(0, 66)
            # print("test data len", len(data))
            row = data[index]
            label = [int(row[37])]
            y.append(label)
            row = np.delete(row, 37)
            x.append(row)
            ran.append(index)
    # print(label)
    # print("lenx:",len(x))
    # print("len x1:",len(x[1]))
    # x = tf.reshape(x, shape=[batch_size, size])
    # label = tf.reshape(label, shape=[batch_size, 2])

    return x, y, ran


def getWeight(shape):
    w = tf.truncated_normal(shape, 0.1)
    return tf.Variable(w)


def getBias(shape):
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)


def fc(input, keep_prob):
    fc1_w = getWeight([size, 128])
    fc1_b = getBias([128])
    fc1 = tf.nn.relu(tf.matmul(input, fc1_w) + fc1_b)
    h_fc1_drop = tf.nn.dropout(fc1, keep_prob)
    fc2_w = getWeight([128, 2])
    fc2_b = getBias([2])
    fc2 = tf.matmul(h_fc1_drop, fc2_w) + fc2_b

    return fc2, fc1_w, fc2_w


def cross_entropy(fc_result, label):
    label = tf.one_hot(label, depth=num_class)
    return tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=fc_result, labels=label))


def train():
    x_ = tf.placeholder(shape=[batch_size, size], dtype="float")
    y_ = tf.placeholder(shape=[batch_size, 1], dtype="int32")
    keep_prob = tf.placeholder("float")

    global_step = tf.Variable(0, name="global_step", trainable=False)
    logits, fc1, fc2 = fc(x_, keep_prob)
    loss = cross_entropy(logits, y_)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=config)
    sess.run(init)
    saver = tf.train.Saver(tf.global_variables())
    whole_data = readData(train_path)
    test_data = readData(test_path)
    # whole_data = sess.run(read_data)
    for i in range(3000000):
        if i % 100 == 0:
            x, y, indexs = makeData(whole_data, "train")
            test_x, test_y, test_index = makeData(test_data, "test")
            ss_x = StandardScaler()
            ss_y = StandardScaler()

            x = ss_x.fit_transform(x)
            y = ss_y.fit_transform(y)
            test_x = ss_x.transform(test_x)
            test_y = ss_y.transform(test_y)

            # x = tf.reshape(x, shape=[batch_size, size])
            # x = tf.cast(x, dtype=tf.float32)
            train_accuracy = sess.run(accuracy, feed_dict={x_: test_x, y_: test_y, keep_prob: 1})
            lo = sess.run(loss, feed_dict={x_: x, y_: y, keep_prob: 1})
            # f1w = sess.run(fc1, feed_dict={x_: x, y_: y, keep_prob: 1})
            # f2w = sess.run(fc2, feed_dict={x_: x, y_: y, keep_prob: 1})
            # print("f1:",f1w,"\n","f2:",f2w)
            print("step %d, train accuracy %g , loss %g" % (i, train_accuracy, lo))
            # print(indexs)
        if i % 5000 == 0 and i != 0:
            save_path = os.path.join(log_path, "model.ckpt")
            saver.save(sess, save_path, global_step=i)

        sess.run(train_op, feed_dict={x_: x, y_: y, keep_prob: 0.5})


if __name__ == "__main__":
    train()


step 0, train accuracy 0.12 , loss 2779.43
step 100, train accuracy 0.21 , loss 2402.67
step 200, train accuracy 0.11 , loss 2761.96
step 300, train accuracy 0.13 , loss 2513.55
step 400, train accuracy 0.14 , loss 2019.63
step 500, train accuracy 0.07 , loss 2752.37
step 600, train accuracy 0.13 , loss 2520
step 700, train accuracy 0.2 , loss 2378.62
step 800, train accuracy 0.12 , loss 1979.97
step 900, train accuracy 0.08 , loss 2432.3
step 1000, train accuracy 0.12 , loss 2374.79
step 1100, train accuracy 0.15 , loss 2658.12
step 1200, train accuracy 0.22 , loss 1995.58
step 1300, train accuracy 0.09 , loss 2516.21
step 1400, train accuracy 0.08 , loss 2219.25
step 1500, train accuracy 0.07 , loss 2013.54
step 1600, train accuracy 0.21 , loss 1928.89
step 1700, train accuracy 0.19 , loss 2396.27
step 1800, train accuracy 0.13 , loss 1743.3
step 1900, train accuracy 0.17 , loss 2025.44
step 2000, train accuracy 0.13 , loss 1645.76
step 2100, train accuracy 0.2 , loss 1902.69
step 22

step 11000, train accuracy 0.49 , loss 1387.63
step 11100, train accuracy 0.43 , loss 920.22
step 11200, train accuracy 0.46 , loss 1499.28
step 11300, train accuracy 0.42 , loss 471.136
step 11400, train accuracy 0.42 , loss 1080.43
step 11500, train accuracy 0.5 , loss 1334.25
step 11600, train accuracy 0.35 , loss 628.705
step 11700, train accuracy 0.47 , loss 1206.68
step 11800, train accuracy 0.48 , loss 1012.4
step 11900, train accuracy 0.54 , loss 1048.03
step 12000, train accuracy 0.54 , loss 1456.52
step 12100, train accuracy 0.47 , loss 1453.94
step 12200, train accuracy 0.32 , loss 962.301
step 12300, train accuracy 0.33 , loss 845.408
step 12400, train accuracy 0.55 , loss 713.52
step 12500, train accuracy 0.56 , loss 999.848
step 12600, train accuracy 0.38 , loss 713.867
step 12700, train accuracy 0.44 , loss 1654.33
step 12800, train accuracy 0.37 , loss 1072.73
step 12900, train accuracy 0.47 , loss 1297.01
step 13000, train accuracy 0.49 , loss 1453.69
step 13100, train

step 22100, train accuracy 0.56 , loss 883.102
step 22200, train accuracy 0.59 , loss 699.891
step 22300, train accuracy 0.49 , loss 1278.8
step 22400, train accuracy 0.54 , loss 652.624
step 22500, train accuracy 0.53 , loss 541.466
step 22600, train accuracy 0.65 , loss 966.145
step 22700, train accuracy 0.61 , loss 568.312
step 22800, train accuracy 0.47 , loss 764.411
step 22900, train accuracy 0.55 , loss 667.622
step 23000, train accuracy 0.6 , loss 828.387
step 23100, train accuracy 0.46 , loss 543.044
step 23200, train accuracy 0.59 , loss 1313.88
step 23300, train accuracy 0.53 , loss 690.661
step 23400, train accuracy 0.59 , loss 815.504
step 23500, train accuracy 0.36 , loss 943.646
step 23600, train accuracy 0.55 , loss 861.481
step 23700, train accuracy 0.54 , loss 607.913
step 23800, train accuracy 0.53 , loss 760.895
step 23900, train accuracy 0.69 , loss 1111.04
step 24000, train accuracy 0.63 , loss 1144.64
step 24100, train accuracy 0.54 , loss 663.043
step 24200, tra

step 33200, train accuracy 0.63 , loss 353.465
step 33300, train accuracy 0.67 , loss 1410.94
step 33400, train accuracy 0.73 , loss 1205.67
step 33500, train accuracy 0.61 , loss 477.425
step 33600, train accuracy 0.63 , loss 945.964
step 33700, train accuracy 0.6 , loss 948.043
step 33800, train accuracy 0.69 , loss 562.641
step 33900, train accuracy 0.63 , loss 349.193
step 34000, train accuracy 0.64 , loss 1110.74
step 34100, train accuracy 0.78 , loss 416.421
step 34200, train accuracy 0.78 , loss 319.321
step 34300, train accuracy 0.54 , loss 466.045
step 34400, train accuracy 0.65 , loss 318.167
step 34500, train accuracy 0.68 , loss 1256.47
step 34600, train accuracy 0.65 , loss 336.565
step 34700, train accuracy 0.66 , loss 521.044
step 34800, train accuracy 0.67 , loss 259.534
step 34900, train accuracy 0.65 , loss 1069.14
step 35000, train accuracy 0.58 , loss 303.142
step 35100, train accuracy 0.56 , loss 196.392
step 35200, train accuracy 0.71 , loss 505.294
step 35300, tr

step 44300, train accuracy 0.73 , loss 231.439
step 44400, train accuracy 0.77 , loss 984.981
step 44500, train accuracy 0.62 , loss 990.386
step 44600, train accuracy 0.72 , loss 290.09
step 44700, train accuracy 0.65 , loss 238.929
step 44800, train accuracy 0.74 , loss 1055.49
step 44900, train accuracy 0.67 , loss 206.561
step 45000, train accuracy 0.58 , loss 1174.78
step 45100, train accuracy 0.76 , loss 948.54
step 45200, train accuracy 0.7 , loss 532.212
step 45300, train accuracy 0.81 , loss 1223.44
step 45400, train accuracy 0.77 , loss 1081.83
step 45500, train accuracy 0.65 , loss 1006.78
step 45600, train accuracy 0.66 , loss 186.931
step 45700, train accuracy 0.59 , loss 325.195
step 45800, train accuracy 0.8 , loss 1069.27
step 45900, train accuracy 0.66 , loss 212.947
step 46000, train accuracy 0.72 , loss 358.876
step 46100, train accuracy 0.69 , loss 163.636
step 46200, train accuracy 0.66 , loss 843.715
step 46300, train accuracy 0.77 , loss 253.41
step 46400, train 

...

step 110500, train accuracy 0.73 , loss 526.079
step 110600, train accuracy 0.69 , loss 486.523
step 110700, train accuracy 0.71 , loss 619.881
step 110800, train accuracy 0.77 , loss 312.271
step 110900, train accuracy 0.67 , loss 581.522
step 111000, train accuracy 0.69 , loss 248.822
step 111100, train accuracy 0.74 , loss 662.753
step 111200, train accuracy 0.74 , loss 244.469
step 111300, train accuracy 0.69 , loss 691.844
step 111400, train accuracy 0.65 , loss 277.298
step 111500, train accuracy 0.73 , loss 212.965
step 111600, train accuracy 0.75 , loss 341.653
step 111700, train accuracy 0.76 , loss 255.643
step 111800, train accuracy 0.69 , loss 598.281
step 111900, train accuracy 0.68 , loss 270.219
step 112000, train accuracy 0.77 , loss 600.157
step 112100, train accuracy 0.77 , loss 303.116
step 112200, train accuracy 0.71 , loss 591.601
step 112300, train accuracy 0.75 , loss 295.647
step 112400, train accuracy 0.76 , loss 814.475
step 112500, train accuracy 0.72 , loss 

step 121300, train accuracy 0.82 , loss 548.827
step 121400, train accuracy 0.81 , loss 256.477
step 121500, train accuracy 0.81 , loss 218.633
step 121600, train accuracy 0.79 , loss 140.042
step 121700, train accuracy 0.74 , loss 461.201
step 121800, train accuracy 0.76 , loss 508.122
step 121900, train accuracy 0.7 , loss 234.475
step 122000, train accuracy 0.75 , loss 575.333
step 122100, train accuracy 0.77 , loss 621.341
step 122200, train accuracy 0.81 , loss 307.197
step 122300, train accuracy 0.72 , loss 594.543
step 122400, train accuracy 0.67 , loss 249.053
step 122500, train accuracy 0.81 , loss 225.737
step 122600, train accuracy 0.78 , loss 132.881
step 122700, train accuracy 0.83 , loss 171.078
step 122800, train accuracy 0.7 , loss 442.877
step 122900, train accuracy 0.74 , loss 200.571
step 123000, train accuracy 0.74 , loss 192.581
step 123100, train accuracy 0.72 , loss 591.077
step 123200, train accuracy 0.77 , loss 221.444
step 123300, train accuracy 0.82 , loss 54

step 132000, train accuracy 0.72 , loss 510.213
step 132100, train accuracy 0.77 , loss 563.5
step 132200, train accuracy 0.75 , loss 690.323
step 132300, train accuracy 0.71 , loss 231.217
step 132400, train accuracy 0.7 , loss 219.37
step 132500, train accuracy 0.71 , loss 478.362
step 132600, train accuracy 0.76 , loss 236.376
step 132700, train accuracy 0.74 , loss 448.895
step 132800, train accuracy 0.68 , loss 549.554
step 132900, train accuracy 0.8 , loss 162.135
step 133000, train accuracy 0.67 , loss 537.246
step 133100, train accuracy 0.76 , loss 324.705
step 133200, train accuracy 0.79 , loss 691.608
step 133300, train accuracy 0.74 , loss 634.437
step 133400, train accuracy 0.69 , loss 406.226
step 133500, train accuracy 0.76 , loss 185.838
step 133600, train accuracy 0.82 , loss 267.843
step 133700, train accuracy 0.69 , loss 485.338
step 133800, train accuracy 0.71 , loss 156.908
step 133900, train accuracy 0.78 , loss 151.185
step 134000, train accuracy 0.73 , loss 264.2

step 142800, train accuracy 0.84 , loss 199.07
step 142900, train accuracy 0.78 , loss 267.057
step 143000, train accuracy 0.67 , loss 207.531
step 143100, train accuracy 0.69 , loss 570.487
step 143200, train accuracy 0.74 , loss 579.976
step 143300, train accuracy 0.65 , loss 261.186
step 143400, train accuracy 0.84 , loss 514.636
step 143500, train accuracy 0.71 , loss 164.868
step 143600, train accuracy 0.78 , loss 535.328
step 143700, train accuracy 0.78 , loss 255.901
step 143800, train accuracy 0.66 , loss 195.737
step 143900, train accuracy 0.73 , loss 263.075
step 144000, train accuracy 0.84 , loss 564.833
step 144100, train accuracy 0.78 , loss 114.266
step 144200, train accuracy 0.73 , loss 526.768
step 144300, train accuracy 0.74 , loss 236.317
step 144400, train accuracy 0.77 , loss 551.556
step 144500, train accuracy 0.74 , loss 251.822
step 144600, train accuracy 0.7 , loss 493.171
step 144700, train accuracy 0.75 , loss 509.823
step 144800, train accuracy 0.69 , loss 27

KeyboardInterrupt: ignored

In [8]:
!ls

datalab


In [9]:
cd log

/content/log


In [10]:
!ls

checkpoint			       model.ckpt-140000.index
model.ckpt-130000.data-00000-of-00001  model.ckpt-140000.meta
model.ckpt-130000.index		       model.ckpt-145000.data-00000-of-00001
model.ckpt-130000.meta		       model.ckpt-145000.index
model.ckpt-135000.data-00000-of-00001  model.ckpt-145000.meta
model.ckpt-135000.index		       model.ckpt-150000.data-00000-of-00001
model.ckpt-135000.meta		       model.ckpt-150000.index
model.ckpt-140000.data-00000-of-00001  model.ckpt-150000.meta


In [7]:
!ls

datalab


In [9]:
from google.colab import files

uploaded = files.upload()


Saving wang_data.csv to wang_data.csv
Saving wang_data_test.csv to wang_data_test.csv


In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import os
from sklearn import preprocessing

num_class = 2
batch_size = 20
train_path = r"wang_data.csv"
test_path = r"wang_data_test.csv"
learning_rate = 0.00001
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
size = 37
log_path = r"./log/"


def readData(path):
    whole_data = pd.read_csv(path, encoding='utf-8')
    whole_data = whole_data.fillna(0)
    whole_list = np.array(whole_data)
    for i in range(len(whole_list)):
        for temp in range(len(whole_list[i])):
            if whole_list[i][temp] == " ":
                whole_list[i][temp] = 0
    # print("转换前：", whole_list)
    whole_list = np.array(whole_list, dtype="float")
    # print(whole_list)
    return whole_list


def makeData(data, flag):
    x = []
    y = []
    ran = []
    # print(len(data))
    # print(len(data[1480]))
    # print(data)
    if flag == "train":
        for i in range(batch_size):
            index = random.randint(0, 6384)
            # print("test data len", len(data))
            row = data[index]
            label = [int(row[37])]
            y.append(label)
            row = np.delete(row, 37)
            x.append(row)
            ran.append(index)
    else:
        for i in range(batch_size):
            index = random.randint(0, 66)
            # print("test data len", len(data))
            row = data[index]
            label = [int(row[37])]
            y.append(label)
            row = np.delete(row, 37)
            x.append(row)
            ran.append(index)
    # print(label)
    # print("lenx:",len(x))
    # print("len x1:",len(x[1]))
    # x = tf.reshape(x, shape=[batch_size, size])
    # label = tf.reshape(label, shape=[batch_size, 2])

    return x, y, ran


def getWeight(shape):
    w = tf.truncated_normal(shape, 0.1)
    return tf.Variable(w)


def getBias(shape):
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)


def fc(input, keep_prob):
    fc1_w = getWeight([size, 128])
    fc1_b = getBias([128])
    fc1 = tf.nn.relu(tf.matmul(input, fc1_w) + fc1_b)
    h_fc1_drop = tf.nn.dropout(fc1, keep_prob)
    fc2_w = getWeight([128, 2])
    fc2_b = getBias([2])
    fc2 = tf.matmul(h_fc1_drop, fc2_w) + fc2_b

    return fc2, fc1_w, fc2_w


def cross_entropy(fc_result, label):
    label = tf.one_hot(label, depth=num_class)
    return tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=fc_result, labels=label))


def train():
    x_ = tf.placeholder(shape=[batch_size, size], dtype="float")
    y_ = tf.placeholder(shape=[batch_size, 1], dtype="int32")
    keep_prob = tf.placeholder("float")

    global_step = tf.Variable(0, name="global_step", trainable=False)
    logits, fc1, fc2 = fc(x_, keep_prob)
    loss = cross_entropy(logits, y_)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    init = tf.global_variables_initializer()
    sess = tf.Session(config=config)
    sess.run(init)
    saver = tf.train.Saver(tf.global_variables())
    whole_data = readData(train_path)
    test_data = readData(test_path)
    # whole_data = sess.run(read_data)
    for i in range(3000000):
        if i % 100 == 0:
            x, y, indexs = makeData(whole_data, "train")
            test_x, test_y, test_index = makeData(test_data, "test")

            #归一化
            min_max_scaler = preprocessing.MinMaxScaler()
            x = min_max_scaler.fit_transform(x)

            # 标准化
            ss_x = StandardScaler()
            ss_y = StandardScaler()
            x = ss_x.fit_transform(x)
            y = ss_y.fit_transform(y)
            #print("x:", x)
            test_x = ss_x.transform(test_x)
            #print("test x:", x)
            test_y = ss_y.transform(test_y)




            # x = tf.reshape(x, shape=[batch_size, size])
            # x = tf.cast(x, dtype=tf.float32)
            train_accuracy = sess.run(accuracy, feed_dict={x_: test_x, y_: test_y, keep_prob: 1})
            lo = sess.run(loss, feed_dict={x_: x, y_: y, keep_prob: 1})
            # f1w = sess.run(fc1, feed_dict={x_: x, y_: y, keep_prob: 1})
            # f2w = sess.run(fc2, feed_dict={x_: x, y_: y, keep_prob: 1})
            # print("f1:",f1w,"\n","f2:",f2w)
            print("step %d, train accuracy %g , loss %g" % (i, train_accuracy, lo))
            # print(indexs)
        if i % 5000 == 0 and i != 0:
            save_path = os.path.join(log_path, "model.ckpt")
            saver.save(sess, save_path, global_step=i)

        sess.run(train_op, feed_dict={x_: x, y_: y, keep_prob: 0.4})


if __name__ == "__main__":
    train()


step 0, train accuracy 0.3 , loss 158.789
step 100, train accuracy 0 , loss 279.095
step 200, train accuracy 0.4 , loss 151.713
step 300, train accuracy 0.4 , loss 160.283
step 400, train accuracy 0.5 , loss 44.5085
step 500, train accuracy 0.25 , loss 216.754
step 600, train accuracy 0.4 , loss 112.508
step 700, train accuracy 0.6 , loss 60.7116
step 800, train accuracy 0.6 , loss 286.789
step 900, train accuracy 0.6 , loss 389.322
step 1000, train accuracy 0.75 , loss 131.933
step 1100, train accuracy 0.65 , loss 56.9048
step 1200, train accuracy 0.55 , loss 121.206
step 1300, train accuracy 0.65 , loss 150.766
step 1400, train accuracy 0.55 , loss 88.835
step 1500, train accuracy 0.8 , loss 140.348
step 1600, train accuracy 0.5 , loss 119.979
step 1700, train accuracy 0.75 , loss 75.5894
step 1800, train accuracy 0.75 , loss 192.416
step 1900, train accuracy 0.65 , loss 157.665
step 2000, train accuracy 0.75 , loss 273.776
step 2100, train accuracy 0.7 , loss 74.6349
step 2200, trai

step 8300, train accuracy 0.7 , loss 126.751
step 8400, train accuracy 0.7 , loss 60.053
step 8500, train accuracy 0.65 , loss 81.7669
step 8600, train accuracy 0.55 , loss 52.3967
step 8700, train accuracy 0.5 , loss 25.8524
step 8800, train accuracy 0.8 , loss 49.0759
step 8900, train accuracy 0.45 , loss 53.7114
step 9000, train accuracy 0.7 , loss 60.0229
step 9100, train accuracy 0.35 , loss 42.4289
step 9200, train accuracy 0.65 , loss 38.3099
step 9300, train accuracy 0.9 , loss 22.3229
step 9400, train accuracy 0.95 , loss 10.0543
step 9500, train accuracy 0.7 , loss 12.1124
step 9600, train accuracy 0.55 , loss 127.316
step 9700, train accuracy 0.6 , loss 46.2457
step 9800, train accuracy 0.75 , loss 37.2549
step 9900, train accuracy 0.75 , loss 89.723
step 10000, train accuracy 0.55 , loss 61.1568
step 10100, train accuracy 0.75 , loss 67.1887
step 10200, train accuracy 0.9 , loss 9.03643
step 10300, train accuracy 0.55 , loss 11.0619
step 10400, train accuracy 0.85 , loss 4.

step 16000, train accuracy 0.05 , loss 53.14
step 16100, train accuracy 0.7 , loss 19.0423
step 16200, train accuracy 0.6 , loss 50.3919
step 16300, train accuracy 0.85 , loss 17.185
step 16400, train accuracy 0.8 , loss 48.7692
step 16500, train accuracy 0.6 , loss 41.6118
step 16600, train accuracy 0.8 , loss 21.5313
step 16700, train accuracy 0.75 , loss 53.7558
step 16800, train accuracy 0 , loss 53.7494
step 16900, train accuracy 0.6 , loss 27.2012
step 17000, train accuracy 0.45 , loss 31.2228
step 17100, train accuracy 0.35 , loss 21.2704
step 17200, train accuracy 0.35 , loss 1.87574
step 17300, train accuracy 0.35 , loss 48.8713
step 17400, train accuracy 0.7 , loss 12.7957
step 17500, train accuracy 0.9 , loss 18.1474
step 17600, train accuracy 0.7 , loss 40.8033
step 17700, train accuracy 0.9 , loss 75.2539
step 17800, train accuracy 0.5 , loss 39.0319
step 17900, train accuracy 0.65 , loss 14.2514
step 18000, train accuracy 0.75 , loss 38.146
step 18100, train accuracy 0.8 

step 23900, train accuracy 0.7 , loss 14.4206
step 24000, train accuracy 0.8 , loss 25.9849
step 24100, train accuracy 0.65 , loss 21.3366
step 24200, train accuracy 0.75 , loss 5.99061
step 24300, train accuracy 0.9 , loss 5.84928
step 24400, train accuracy 1 , loss 26.658
step 24500, train accuracy 0 , loss 20.4145
step 24600, train accuracy 0.3 , loss 32.6322
step 24700, train accuracy 1 , loss 19.0693
step 24800, train accuracy 0 , loss 57.9257
step 24900, train accuracy 1 , loss 17.9827
step 25000, train accuracy 1 , loss 7.56997
step 25100, train accuracy 0.95 , loss 25.7983
step 25200, train accuracy 0.9 , loss 9.19616
step 25300, train accuracy 1 , loss 20.5459
step 25400, train accuracy 0 , loss 61.3153
step 25500, train accuracy 1 , loss 10.4643
step 25600, train accuracy 1 , loss 15.8911
step 25700, train accuracy 1 , loss 14.4646
step 25800, train accuracy 1 , loss 19.2644
step 25900, train accuracy 0 , loss 1.58976
step 26000, train accuracy 0 , loss 67.9522
step 26100, tr

step 32000, train accuracy 0 , loss 2.97706
step 32100, train accuracy 0 , loss 14.7248
step 32200, train accuracy 0.05 , loss 5.38193
step 32300, train accuracy 0 , loss 19.8964
step 32400, train accuracy 1 , loss 13.3844
step 32500, train accuracy 0 , loss 22.5102
step 32600, train accuracy 1 , loss 12.0821
step 32700, train accuracy 0.75 , loss 9.07271
step 32800, train accuracy 0 , loss 21.2662
step 32900, train accuracy 1 , loss 4.04138
step 33000, train accuracy 1 , loss 14.7076
step 33100, train accuracy 0 , loss 20.0013
step 33200, train accuracy 0.8 , loss 6.64015
step 33300, train accuracy 1 , loss 4.30874
step 33400, train accuracy 1 , loss 4.93872
step 33500, train accuracy 1 , loss 2.14776
step 33600, train accuracy 1 , loss 1.13331
step 33700, train accuracy 1 , loss 2.34311
step 33800, train accuracy 1 , loss 1.40912
step 33900, train accuracy 1 , loss 21.2182
step 34000, train accuracy 0.15 , loss 6.03259
step 34100, train accuracy 0 , loss 20.3438
step 34200, train acc

step 40000, train accuracy 0 , loss 18.0646
step 40100, train accuracy 0 , loss 8.99203
step 40200, train accuracy 1 , loss 13.3478
step 40300, train accuracy 0 , loss 7.08108
step 40400, train accuracy 0 , loss 13.0485
step 40500, train accuracy 0 , loss 6.10743
step 40600, train accuracy 0 , loss 4.91344
step 40700, train accuracy 0 , loss 17.8259
step 40800, train accuracy 1 , loss 9.695
step 40900, train accuracy 0 , loss 15.5202
step 41000, train accuracy 1 , loss 4.12751
step 41100, train accuracy 1 , loss 2.69297
step 41200, train accuracy 1 , loss 21.8873
step 41300, train accuracy 0 , loss 15.8676
step 41400, train accuracy 0 , loss 5.80953
step 41500, train accuracy 0 , loss 3.54557
step 41600, train accuracy 0 , loss 8.21063
step 41700, train accuracy 0 , loss 14.0074
step 41800, train accuracy 0 , loss 10.0266


KeyboardInterrupt: ignored

In [11]:
!ls

datalab  wang_data.csv	wang_data_test.csv
